In [ ]:
#python 3.13.9
import os
from ultralytics import YOLO
import torch
print("✅ torch импортирован")
print("Версия:", torch.__version__)
print("GPU доступен:", torch.cuda.is_available())
print("Тип device:", type(torch.device('cpu')))

data_yaml_path = r"megadataset/data.yaml"


if not os.path.exists(data_yaml_path):
    raise FileNotFoundError(f"Файл {data_yaml_path} не найден. Убедитесь, что структура папок верна.")

print(f"Используется конфиг: {data_yaml_path}")


model = YOLO('runs/detect/cps5/weights/best.pt') # ГОТОВАЯ МОДЕЛЬ
from ultralytics import YOLO

#model = YOLO('yolov8n.yaml')  # или yolov8n.pt АРХИТЕКТУРА

model.train(
    data=data_yaml_path,
    epochs=60,
    imgsz=640,
    batch=8,
    lr0=0.001,
    lrf=0.01,
    warmup_epochs=0,
    optimizer='AdamW',
    patience=50,
    cos_lr=True,
    name='cps',
    device='0',
    # Аугментации — включаем и расширяем (см. ниже)
    augment=True,        # включить стандартные аугментации YOLO
    mosaic=0.5,          # % батчей — Mosaic (4 изображения в одном)
    mixup=0.25,          # % батчей — MixUp (смешивание изображений)
    degrees=6.0,        # вращение
    translate=0.1,       # сдвиг на % ширины/высоты
    scale=0.2,           # масштаб ±% 
    shear=2.0,           # наклон (shear) ±2°
    perspective=0.001,   # лёгкая перспектива
    flipud=0.0,          # вертикальный флип
    fliplr=0.5,          # горизонтальный флип — 50%
    hsv_h=0.015,         # оттенок (hue) ±1.5%
    hsv_s=0.5,           # насыщенность (saturation) ±70% (Roboflow даёт ±61% → добавляем запас)
    hsv_v=0.4,           # яркость (value) ±40% (Roboflow ±25% → усиливаем)
    cache=False          # если RAM мало — поставьте True (но медленнее)
)

# 4. Валидация на test-наборе
metrics = model.val(data=data_yaml_path, split='test')
print("✅ Обучение завершено!")
print(f"mAP50-95 на тестовом наборе: {metrics.box.map:.4f}")

✅ torch импортирован
Версия: 2.9.1+cu130
GPU доступен: True
Тип device: <class 'torch.device'>
Используется конфиг: megadataset/data.yaml
New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.223  Python-3.13.9 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=megadataset/data.yaml, degrees=6.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_

## **Создание датасета из датасетов**

In [ ]:
import os
import shutil
from pathlib import Path
import yaml

def merge_datasets(dataset_paths, output_path):
    """
    Объединяет несколько YOLO-датасетов в один.
    
    :param dataset_paths: список путей к папкам датасетов (каждая содержит data.yaml, train/, valid/, test/)
    :param output_path: путь к выходной папке (например, 'megadataset')
    """

    all_classes = set()
    class_mapping = {}  # {old_class_name: new_class_id}
    class_names = []    # [name0, name1, ...] — порядок по мере появления

    for ds_path in dataset_paths:
        yaml_path = Path(ds_path) / 'data.yaml'
        with open(yaml_path, 'r', encoding='utf-8') as f:
            cfg = yaml.safe_load(f)
        names = cfg['names']
        for name in names:
            if name not in all_classes:
                all_classes.add(name)
                class_names.append(name)
                class_mapping[name] = len(class_names) - 1  # 0-based index

    print(f"✅ Найдено {len(all_classes)} уникальных классов:")
    for i, name in enumerate(class_names):
        print(f"   {i}: {name}")

    output_path = Path(output_path)
    output_path.mkdir(parents=True, exist_ok=True)

    splits = ['train', 'valid', 'test']

    for split in splits:
        img_out = output_path / split / 'images'
        lbl_out = output_path / split / 'labels'
        img_out.mkdir(parents=True, exist_ok=True)
        lbl_out.mkdir(parents=True, exist_ok=True)

    for ds_idx, ds_path in enumerate(dataset_paths):
        ds_path = Path(ds_path)
        print(f"\n🔄 Обработка датасета {ds_idx+1}/{len(dataset_paths)}: {ds_path.name}")

        with open(ds_path / 'data.yaml', 'r', encoding='utf-8') as f:
            cfg = yaml.safe_load(f)
        old_names = cfg['names']

        for split in splits:
            src_img = ds_path / split / 'images'
            src_lbl = ds_path / split / 'labels'

            if not src_img.exists():
                continue

            for img_file in src_img.iterdir():
                new_name = f"{ds_path.name}_{img_file.name}"
                dst_img = output_path / split / 'images' / new_name
                shutil.copy(img_file, dst_img)


                lbl_file = src_lbl / (img_file.stem + '.txt')
                if lbl_file.exists():
                    with open(lbl_file, 'r', encoding='utf-8') as f:
                        lines = f.readlines()

                    new_lines = []
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) < 5:
                            continue

                        old_cls_id = int(parts[0])
                        old_cls_name = old_names[old_cls_id]
                        new_cls_id = class_mapping[old_cls_name]

                        new_line = f"{new_cls_id} {' '.join(parts[1:])}\n"
                        new_lines.append(new_line)

                    dst_lbl = output_path / split / 'labels' / (new_name.replace('.jpg', '.txt').replace('.png', '.txt'))
                    with open(dst_lbl, 'w', encoding='utf-8') as f:
                        f.writelines(new_lines)

    data_yaml_content = {
        'train': '../train/images',
        'val': '../valid/images',
        'test': '../test/images',
        'nc': len(class_names),
        'names': class_names,
        'roboflow': {
            'workspace': 'merged',
            'project': 'megadataset',
            'version': 1,
            'license': 'CC BY 4.0',
            'url': 'https://example.com/megadataset'
        }
    }

    with open(output_path / 'data.yaml', 'w', encoding='utf-8') as f:
        yaml.dump(data_yaml_content, f, default_flow_style=False, allow_unicode=True)

    print(f"\n🎉 Датасет успешно объединён в: {output_path}")
    print(f"   Всего классов: {len(class_names)}")
    print(f"   Файл data.yaml создан.")

datasets = [
        'datasets/carsandpedestrians',
        'datasets/pedestriansandcars',
        'datasets/russiansigns'
    ]
output_dir = 'megadataset'

merge_datasets(datasets, output_dir)

✅ Найдено 58 уникальных классов:
   0: bicycle
   1: bus
   2: car
   3: motorcycle
   4: pedestrian
   5: scooter
   6: shuttle_taxi
   7: truck
   8: 100Limit
   9: 10Limit
   10: OneWayRoad
   11: OneWayRoadRight
   12: 20Limit
   13: 30Limit
   14: 40Limit
   15: 50Limit
   16: 5Limit
   17: 60Limit
   18: 70Limit
   19: 80Limit
   20: BusRoad
   21: BusStop
   22: Children
   23: Forbidden
   24: Force
   25: ForwardAndLeftWay
   26: ForwardAndRightWay
   27: ForwardWay
   28: Information
   29: LeftAndRightWay
   30: LeftWay
   31: NoPark
   32: NoParkAndStop
   33: NoRight
   34: Parking
   35: Crossroad
   36: RightWay
   37: RingRoad
   38: Special
   39: Stop
   40: Tram
   41: DeadEnd
   42: GiveWay
   43: Warning
   44: OneWayRoadEnd
   45: MainRoadEnd
   46: MainRoad
   47: NoLeft
   48: ProhibitorySign
   49: PassLeft
   50: PassLeftRight
   51: PassRight
   52: OnlyForward
   53: OnlyForwardAndLeft
   54: OnlyForwardAndRight
   55: OnlyLeft
   56: OnlyRight
   57: OnlyRi

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Путь к results.csv (адаптируйте при необходимости)
csv_path = 'runs/detect/cps6/results.csv'
df = pd.read_csv(csv_path)

# Убедимся, что столбцы называются как в YOLOv8 (иногда бывают train/box_loss или val/box_loss)
# Используем первые доступные столбцы с нужными подстроками
box_loss_col = [c for c in df.columns if 'train/box_loss' in c][0]
cls_loss_col = [c for c in df.columns if 'train/cls_loss' in c][0]
dfl_loss_col = [c for c in df.columns if 'train/dfl_loss' in c][0]

# Строим график
fig, ax1 = plt.subplots(figsize=(10, 6))

# Лоссы
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='black')
ax1.plot(df['epoch'], df[box_loss_col], label='Box Loss', color='steelblue', linewidth=2)
ax1.plot(df['epoch'], df[cls_loss_col], label='Class Loss', color='darkorange', linewidth=2)
ax1.plot(df['epoch'], df[dfl_loss_col], label='DFL Loss', color='seagreen', linewidth=2)
ax1.tick_params(axis='y')
ax1.grid(True, alpha=0.3)

# Вторая ось — mAP
ax2 = ax1.twinx()
mAP50_col = [c for c in df.columns if 'mAP50(B)' in c][0]
mAP50_95_col = [c for c in df.columns if 'mAP50-95(B)' in c][0]

ax2.set_ylabel('mAP', color='crimson')
ax2.plot(df['epoch'], df[mAP50_col], label='mAP50', color='crimson', linestyle='--', linewidth=2)
ax2.plot(df['epoch'], df[mAP50_95_col], label='mAP50-95', color='purple', linestyle='--', linewidth=2)
ax2.tick_params(axis='y', labelcolor='crimson')

# Легенда и заголовок
fig.suptitle('YOLOv8 Training: Loss & mAP', fontsize=14)
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=5, frameon=True)
plt.tight_layout()

# Сохраняем в PDF
output_pdf = 'training_metrics5.pdf'
plt.savefig(output_pdf, format='pdf', bbox_inches='tight', dpi=300)
plt.close()  # освобождаем память

print(f"✅ График сохранён в: {output_pdf}")

✅ График сохранён в: training_metrics5.pdf


## **Код для графика PR**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

csv_path = 'runs/detect/cps4/results.csv'
df = pd.read_csv(csv_path)

plt.figure(figsize=(10, 6))

plt.plot(df['epoch'], df['metrics/precision(B)'], label='Precision', color='teal', linewidth=2, marker='o', markersize=3)
plt.plot(df['epoch'], df['metrics/recall(B)'], label='Recall', color='magenta', linewidth=2, marker='s', markersize=3)

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Precision and Recall over Epochs', fontsize=14)
plt.grid(True, alpha=0.4)
plt.legend(loc='lower right', fontsize=12)
plt.ylim(0.0, 1.05)  # Precision и Recall ∈ [0, 1]

output_pdf = 'precision_recall5.pdf'
plt.tight_layout()
plt.savefig(output_pdf, format='pdf', bbox_inches='tight', dpi=300)
plt.close()

print(f"✅ Precision & Recall график сохранён в: {output_pdf}")

✅ Precision & Recall график сохранён в: precision_recall5.pdf


## **Трекинг и рамки**

In [ ]:
import os
import cv2
from ultralytics import YOLO

# Загрузка модели
model = YOLO('runs/detect/cps5/weights/best.pt')

detected_objects = {}

# Путь к видео
video_path = "sings.mp4"
output_path = "output_tracked.mp4"

# Настройки трекинга
results = model.track(
    source=video_path,
    tracker="bytetrack.yaml",
    persist=True,
    conf=0.5,
    iou=0.7,
    show=False,        
    stream=True     
)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

class_colors = {
    0: (0, 255, 0), 
    1: (0, 0, 255),   
    2: (255, 0, 0),    
    3: (255, 255, 0),
}

# Проходим по каждому кадру
frame_id = 0
for result in results:
    frame_id += 1
    timestamp_sec = frame_id / fps
    

    frame = result.orig_img.copy()

    if result.boxes.id is not None:
        ids = result.boxes.id.cpu().numpy().astype(int)
        cls_ids = result.boxes.cls.cpu().numpy().astype(int)
        confs = result.boxes.conf.cpu().numpy()
        xyxy = result.boxes.xyxy.cpu().numpy()
        
        frame_detected_objects = []

        for i in range(len(ids)):
            track_id = ids[i]
            cls_id = cls_ids[i]
            conf = confs[i]
            x1, y1, x2, y2 = map(int, xyxy[i])


            # Цвет рамки по классу
            color = class_colors.get(cls_id, (255, 255, 255))  # белый по умолчанию

            # Рисуем прямоугольник
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Подпись: "Class ID:conf"
            label = f"{model.names[cls_id]} {track_id} ({conf:.2f})"
            font_scale = 0.6
            thickness = 2
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
            
            frame_detected_objects.append({
                    'id': track_id,
                    'class': model.names[cls_id],
                    'conf': conf,
                    'bbox': (x1, y1, x2, y2)
            })

            # Фон для текста
            cv2.rectangle(frame, (x1, y1 - text_height - 5), (x1 + text_width + 5, y1), color, -1)
            cv2.putText(frame, label, (x1 + 2, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), thickness)

            # Лог в консоль (опционально)
            # print(f"[{frame_id:04d} | {timestamp_sec:6.2f}s] "
            #       f"ID={track_id:3d} | {model.names[cls_id]:10s} | conf={conf:.2f} | box=({x1},{y1})–({x2},{y2})")
        
        detected_objects[frame_id] = frame_detected_objects

    else:
        print(f"[{frame_id:04d}] — no tracked objects")

    # Записываем кадр в видео
    out.write(frame)

# Закрываем всё
out.release()
cv2.destroyAllWindows()

print(f"\n✅ Видео сохранено: {output_path}")

detected_objects


video 1/1 (frame 1/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 61.2ms
video 1/1 (frame 2/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 41.5ms
video 1/1 (frame 3/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 20.3ms
video 1/1 (frame 4/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 18.5ms
video 1/1 (frame 5/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 37.2ms
video 1/1 (frame 6/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 10.6ms
video 1/1 (frame 7/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 8.9ms
video 1/1 (frame 8/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 8.7ms
video 1/1 (frame 9/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 1 car, 7.4ms
video 1/1 (frame 10/1241) c:\Users\maksg\Desktop\thatsiencebitch\sings.mp4: 640x384 (no detections), 10.5ms
[0010] — no tracked o

TypeError: Object of type int64 is not JSON serializable

In [2]:
import json
import numpy as np

def convert(o):
    if isinstance(o, np.generic):  # np.int32, np.float64 ...
        return o.item()
    if isinstance(o, np.ndarray):
        return o.tolist()
    raise TypeError

with open("totallysecretsciencedata.json", "w", encoding="utf-8") as f:
    json.dump(detected_objects, f, ensure_ascii=False, indent=4, default=convert)